In [81]:
!pip install torchtext==0.4.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [82]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchtext.datasets import IMDB
from torchtext.data import Field, LabelField, BucketIterator

In [83]:
# Set device to use GPU if available, otherwise use CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [84]:
# Define hyperparameters
batch_size = 64
num_epochs = 50
learning_rate = 0.001
hidden_dim = 500

In [85]:
# Define data transforms to preprocess input
TEXT = Field(lower=True, batch_first=True)
LABEL = LabelField(dtype=torch.float)

In [86]:
train_data, test_data = IMDB.splits(TEXT, LABEL)

In [87]:
TEXT.build_vocab(train_data, max_size=5000)
LABEL.build_vocab(train_data)

In [88]:
train_loader, test_loader = BucketIterator.splits(
    (train_data, test_data), batch_size=batch_size, device=device)

In [89]:
# Define neural network architecture
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.embedding = nn.Embedding(len(TEXT.vocab), 100)
        self.fc1 = nn.Linear(100, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, 1)
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        x = self.embedding(x)
        x = torch.mean(x, dim=1)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        x = torch.sigmoid(x)
        return x.squeeze()

In [90]:
# Create neural network and move it to device
net = Net().to(device)

In [91]:
# Define loss function and optimizer
criterion = nn.BCELoss()
optimizer = optim.Adam(net.parameters(), lr=learning_rate)

In [92]:
# Train neural network
for epoch in range(num_epochs):
    running_loss = 0.0
    for i, batch in enumerate(train_loader):
        inputs, labels = batch.text.to(device), batch.label.to(device)

        # Forward pass
        outputs = net(inputs)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Print statistics
        running_loss += loss.item()
        if (i+1) % 100 == 0:
            print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'
                  .format(epoch+1, num_epochs, i+1, len(train_loader), running_loss / 100))
            running_loss = 0.0

Epoch [1/50], Step [100/391], Loss: 0.6969
Epoch [1/50], Step [200/391], Loss: 0.6972
Epoch [1/50], Step [300/391], Loss: 0.6931
Epoch [2/50], Step [100/391], Loss: 0.6859
Epoch [2/50], Step [200/391], Loss: 0.6763
Epoch [2/50], Step [300/391], Loss: 0.6565
Epoch [3/50], Step [100/391], Loss: 0.5609
Epoch [3/50], Step [200/391], Loss: 0.5324
Epoch [3/50], Step [300/391], Loss: 0.4836
Epoch [4/50], Step [100/391], Loss: 0.4195
Epoch [4/50], Step [200/391], Loss: 0.4034
Epoch [4/50], Step [300/391], Loss: 0.3832
Epoch [5/50], Step [100/391], Loss: 0.3691
Epoch [5/50], Step [200/391], Loss: 0.3568
Epoch [5/50], Step [300/391], Loss: 0.3700
Epoch [6/50], Step [100/391], Loss: 0.3373
Epoch [6/50], Step [200/391], Loss: 0.3366
Epoch [6/50], Step [300/391], Loss: 0.3316
Epoch [7/50], Step [100/391], Loss: 0.3242
Epoch [7/50], Step [200/391], Loss: 0.3036
Epoch [7/50], Step [300/391], Loss: 0.3117
Epoch [8/50], Step [100/391], Loss: 0.2957
Epoch [8/50], Step [200/391], Loss: 0.3043
Epoch [8/50

In [94]:
# Define file path to save the trained model
model_path = 'one_layer_FFNN_model.pt'

# Save trained model
torch.save(net.state_dict(), model_path)

In [95]:
# Test neural network
with torch.no_grad():
    correct = 0
    total = 0
    for batch in test_loader:
        inputs, labels = batch.text.to(device), batch.label.to(device)
        outputs = net(inputs)
        predicted = torch.round(outputs)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Test Accuracy of the model on the {} test reviews: {} %'.format(total, 100 * correct / total))

Test Accuracy of the model on the 25000 test reviews: 85.96 %
